# Python: assignment, function argument passing, views, and copies

When writing Python codes, I sometimes get a lot of question marks in my head when applying certain operations on variables: does it change the variable in place? does it return a copy or a reference of the orginal variable? If I change a submatrix, would the changes also be reflected in the original?

To get a better understanding of how things work in Python, I ran a couple of experiments.

## Assignment and function argument passing
**Take home messages**:
- Python **do not make copies** when performing normal assignment, function argument passing, or function returning
- There are two types of variables in Python: **mutable** and **immutable**. Operations like `+=` are inplace for mutables,but return a new object for immutables.

### Assignment

In [1]:
## example 1: float
a = 3.3
b = a
print("id of a:",id(a))
print("id of b:",id(b))

print("change b")
b += 1
print("id of a:",id(a))
print("id of b:",id(b))
print("a is:",a)
print("b is:",b)

id of a: 4546375920
id of b: 4546375920
change b
id of a: 4546375920
id of b: 4546375944
a is: 3.3
b is: 4.3


In the above example, `b` is assigned with float `a`. By checking memory location of `a` and `b`, we can see they both point to the same memory. However, after applying the `+=` operator, `b` points to another location. The change is only effective to `b`, not to `a`.

Next, we experiment with strings:

In [2]:
## example 2: string
a = "string"
b = a
print("id of a:",id(a))
print("id of b:",id(b))

print("change b")
b += "_new"
print("id of a:",id(a))
print("id of b:",id(b))
print("a is:",a)
print("b is:",b)

id of a: 4504620312
id of b: 4504620312
change b
id of a: 4504620312
id of b: 4546869808
a is: string
b is: string_new


Behaviors are similar to those observed from the float example. 

How about list?

In [4]:
## example 3: list
a = [0,1,2]
b = a
print("id of a:",id(a))
print("id of b:",id(b))

print("change b")
b += [0,1,2]
print("id of a:",id(a))
print("id of b:",id(b))
print("a is:",a)
print("b is:",b)


id of a: 4546719176
id of b: 4546719176
change b
id of a: 4546719176
id of b: 4546719176
a is: [0, 1, 2, 0, 1, 2]
b is: [0, 1, 2, 0, 1, 2]


This time we observe different behaviors: after `+=`, address of `b` did not change; the change made to `b` also affected `a`.

The differences in such behaviors are due to the difference between **mutable** and **immutable** variable types. The immutable type, as the name suggests, cannot be modified. Thus operations, such as `+=, -=, ...`, would result in a new object. With mutables, the operations can make changes in place, thus object address remains the same.

### Function argument passing

In [5]:
def f(x):
    print("id in function:",id(x))
    return x

## example 1: int
print("example 1: int ------------")
a = 1
print("id of a:",id(a))
b = f(a)
print("id of the returned value b:",id(b))

### example 2: list
print("example 2: list ------------")
a = [1,2,3]
print("id of a:",id(a))
b = f(a)
print("id of the returned value b:",id(b))

example 1: int ------------
id of a: 4501985328
id in function: 4501985328
id of the returned value b: 4501985328
example 2: list ------------
id of a: 4522021640
id in function: 4522021640
id of the returned value b: 4522021640


In each example above, we print three addresses:
1. address of the variable to be passed into a function
2. address of the variable that the function receives
3. address of the variable returned by the function

All the 3 addresses are the same for both mutable and immutable types. There is no copy of any kind involved in the process of argument passing or function returning.

## More experiments with different Python objects

### Python lists

Take home message:
- subsetting creates a new object
- `listA + listB` creates a new list; `listA += listB` modifies `listA` in place

In [6]:
# list subsetting
print("I. subsetting: ------------")
x = list(range(10))
y = x[:5]
print("id of x:",id(x))
print("id o y:",id(y)) # y is at a different location
print("change y:")
y[0]=-1
print("x is:",x) # x is not affected
print("y is:",y) 

# += vs +
print("II. += vs + ------------")
a = [1,2,3]
print("id of a:",id(a))
b = a + [4]
print("id of b:",id(b)) # different object
a += [1]
print("id of a:",id(a)) # change in place


I. subsetting: ------------
id of x: 4546905608
id o y: 4546718664
change y:
x is: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
y is: [-1, 1, 2, 3, 4]
II. += vs + ------------
id of a: 4545716040
id of b: 4546869960
id of a: 4545716040


For most member functions of list, they modify in place. However, it never harms to check out the function details online, 
if you are not sure how it actually works.

### Numpy array
Take home messages:
- **normal array indexing does not result in a new copy**, but instead a **view**. Changes in the indexed object can also be reflected in the original object
- **fancy indexing (non-consecutive indices) does not return views**
- if you want a copy when choosing submatrix, you need to use specific commands (eg. `array.copy()`)

In [7]:
import numpy as np
A =  np.array(range(9)).reshape(3,3)
print("A is:\n",A)

# indexing
print("\nI. indexing -------------")
B = A[:2,:2]
print("B is a subset of A:\n",B)

print("change B")
B[0] = [-1, -1]
print("B is:\n",B)
print("A is:\n",A) # A is changed

# fancy indexing
print("\nII. fancy indexing -------------")
A =  np.array(range(9)).reshape(3,3)
B = A[[0,2],:][:,[0,2]]
print("B is a subset of A:\n",B)

print("change B")
B[0] = [99, 99]
print("B is:\n",B)
print("A is:\n",A) # A is also changed


# make copy
print("\nIII Make a copy ------------")
A =  np.array(range(9)).reshape(3,3)
B = A[:2,:2].copy()
print("B is a subset of A:\n",B)

print("change B")
B[0] = [-99, -99]
print("B is:\n",B)
print("A is:\n",A) # A not changed

A is:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]

I. indexing -------------
B is a subset of A:
 [[0 1]
 [3 4]]
change B
B is:
 [[-1 -1]
 [ 3  4]]
A is:
 [[-1 -1  2]
 [ 3  4  5]
 [ 6  7  8]]

II. fancy indexing -------------
B is a subset of A:
 [[0 2]
 [6 8]]
change B
B is:
 [[99 99]
 [ 6  8]]
A is:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]

III Make a copy ------------
B is a subset of A:
 [[0 1]
 [3 4]]
change B
B is:
 [[-99 -99]
 [  3   4]]
A is:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]


### Pandas DataFrame
Take home messages:
- Similar to numpy array, **normal indexing in pandas dataframe does not create a copy** either.
- fancy indexing does not return views
- if you want a copy when choosing dataframe, you need to use specific commands (eg. `array.copy()`)

In [8]:
import pandas as pd

In [11]:
A =  pd.DataFrame({'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})
print("A is:\n",A)

# indexing
print("\nI. indexing -------------")
B = A.iloc[:2,:2]
print("B is a subset of A:\n",B)

print("add new column to B")
B['D'] = [99,99]
print("B is:\n",B)
print("A is:\n",A)
print("A does not change\n")

print("change values that shared by A and B")
B['A'] = [99, 99]
print("B is:\n",B)
print("A is:\n",A)
print("A is changed\n")

print("\nII. fancy indexing -------------")
A =  pd.DataFrame({'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})
B = A.iloc[[0,2],[0,2]]
print("B is a subset of A:\n",B)

print("change  B")
B['A'] = [99, 99]
print("B is:\n",B)
print("A is:\n",A)
print("A is not changed\n")


# make copy
print("\nIII Make a copy ------------")
A =  pd.DataFrame({'A':[1,2,3],'B':[4,5,6],'C':[7,8,9]})
B = A.iloc[:2,:2].copy()
print("B is a subset of A:\n",B)
print("\nchange B")
B['A'] = [-99, -99]
print("B is:\n",B)
print("A is:\n",A) # A not changed
print("A is not changed\n")

A is:
    A  B  C
0  1  4  7
1  2  5  8
2  3  6  9

I. indexing -------------
B is a subset of A:
    A  B
0  1  4
1  2  5
add new column to B
B is:
    A  B   D
0  1  4  99
1  2  5  99
A is:
    A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
A does not change

change values that shared by A and B
B is:
     A  B   D
0  99  4  99
1  99  5  99
A is:
     A  B  C
0  99  4  7
1  99  5  8
2   3  6  9
A is changed


II. fancy indexing -------------
B is a subset of A:
    A  C
0  1  7
2  3  9
change  B
B is:
     A  C
0  99  7
2  99  9
A is:
    A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
A is not changed


III Make a copy ------------
B is a subset of A:
    A  B
0  1  4
1  2  5

change B
B is:
     A  B
0 -99  4
1 -99  5
A is:
    A  B  C
0  1  4  7
1  2  5  8
2  3  6  9
A is not changed

